# Calculating Revenue Growth Percentage % using SQL, Pandas and Python3 Functions

## Month-over-Month (MoM) Growth Percentage 

This is a key financial metric that shows the rate at which a value (such as revenue, profit, or customer count) has increased or decreased compared to the previous month.

## Why is it important:

__Short-Term Trend Tracking:__ Month Over Month revenue growth gives a timely snapshot of performance, helping finacial teams understand recent changes without waiting for quarterly or yearly trends.

__Early Detection of Issues or Wins:__ A sharp drop or spike in Month Over Month revenue, expenses, or customer churn can be an early indicator of problems or successes needing immediate attention.

__Credit/Loan Default Volume Metric:__ A Month Over Month revenue growth change in loan defaults or charge-offs. A sudden rise can signal portfolio risk escalation or underwriting weaknesses.

__Credit Utilization Ratios:__ Increasing utilization rates can indicate financial stress among borrowers and signal future repayment problems.

For example if a bank sees a 10% Month Over Month increase in digital loan applications. This might:

* Prompt deeper analysis into customer behavior.

* Signal growing digital engagement.

* Justify additional investment in the mobile platform.




## Method 1: SQL Server CTE to Calculate Revenue Growth Percentage %

```sql
-- month over month revenue growth
-- revenue growth % = (current month revenue - previous month revenue X 100) / previous month revenue

 --Month-over-month revenue growth
with month_rev as (
    select 
        month(order_date) as month,
        coalesce(sum(order_amount),0) as monthly_revenue
    from orders
    group by month(order_date)
)
select 
    month,
    monthly_revenue,
    coalesce(lag(monthly_revenue) over(order by month),0) as previous_month_revenue,
    coalesce(round(monthly_revenue - lag(monthly_revenue) over(order by month) * 100 / 
    nullif(lag(monthly_revenue) over(order by month),0), 2), 0) as revenue_growth_percent
from month_rev;

```


## Method 2: Pandas - Calculate Revenue Growth Percentage %

In [3]:
import pandas as pd

orders = pd.read_csv('/Users/johnryan/Downloads/Documents/Data Sets/orders.csv')
orders.head(5)

,order_id,customer_id,order_date,order_amount
0,1,1006,2023-12-15,405.81
1,2,1016,2023-08-01,953.18
2,3,1019,2023-08-13,745.39
3,4,1003,2023-01-31,618.73
4,5,1004,2023-05-07,198.22


In [4]:
# replicating at the SQL server CTE above to calculate month over month revenue growth
#get the month from order_date and group by month

orders['order_date'] = pd.to_datetime(orders['order_date'])
monthly_revenue = orders.groupby(orders['order_date'].dt.month)['order_amount'].sum().reset_index(name='monthly_revenue')
monthly_revenue.rename(columns={'order_date': 'month'}, inplace=True)



# Now the CTE can be used to help with calcualtion of revenue growth
# Calculate previous month revenue .shift(1) in pandas is the Python equivalent of the SQL LAG() window function

monthly_revenue['previous_month'] = monthly_revenue['monthly_revenue'].shift(1).fillna(0)
monthly_revenue['revenue_growth_percent'] = ((monthly_revenue['monthly_revenue'].fillna(0) - monthly_revenue['previous_month'].fillna(0)) 
/ monthly_revenue['previous_month']).round(1) * 100
monthly_revenue.head()

,month,monthly_revenue,previous_month,revenue_growth_percent
0,1,2051.37,0.00,inf
1,2,3225.72,2051.37,60.0
2,3,2441.02,3225.72,-20.0
3,4,5266.72,2441.02,120.0
4,5,2474.76,5266.72,-50.0


## Method 3: Pandas using function - calculate revenue growth percentage %

In [12]:
#Pandas:

def get_monthly_revenue(orders):
    orders['order_date'] = pd.to_datetime(orders['order_date']) #order date to date time
    orders['month'] = orders['order_date'].dt.month #Extract the month number and create a new 'month' column
    orders = orders.sort_values(by='order_date') #Sort the orders by 'order_date' to ensure correct chronological order
    
    monthly_revenue = (
        orders.groupby('month') #group by month
        .agg(monthly_revenue=('order_amount', 'sum')) #sum of amount to get current reveue
        .reset_index()
    )
    monthly_revenue.rename(columns={'order_date': 'month'}, inplace = True)
    return monthly_revenue


monthly_revenue = get_monthly_revenue(orders)

#calculate the revenue groth percentage

def calculate_revenue(monthly_revenue):
    monthly_revenue['previous_month_revenue'] = monthly_revenue['monthly_revenue'].shift(1).fillna(0) #lag monhtly reveue to get previous month revenue
    monthly_revenue['revenue_growth_percent'] = ((monthly_revenue['monthly_revenue'] -  monthly_revenue['previous_month_revenue'])/
    monthly_revenue['previous_month_revenue']).round(2) *100
    return monthly_revenue

revenue_growth_percent = calculate_revenue(monthly_revenue)
revenue_growth_percent

,month,monthly_revenue,previous_month_revenue,revenue_growth_percent
0,1,2051.37,0.00,inf
1,2,3225.72,2051.37,57.0
2,3,2441.02,3225.72,-24.0
3,4,5266.72,2441.02,116.0
4,5,2474.76,5266.72,-53.0
5,6,5196.00,2474.76,110.0
6,7,5013.02,5196.00,-4.0
7,8,6381.82,5013.02,27.0
8,9,3361.34,6381.82,-47.0
9,10,5551.05,3361.34,65.0


# Create a "Class" for a reusable, modular structure

Keeps all logic related to revenue inside one structure.

Groups related functions under a meaningful name (RevenueAnalyzer).

Easy to add more features later (e.g. year-over-year growth, plotting, etc.)

In [3]:
import pandas as pd
orders = pd.read_csv('/Users/johnryan/Downloads/Documents/Data Sets/orders.csv')

class RevenueAnalyzer:
    def __init__(self, orders):          #__init__ is the constructor: runs when an object is created. Self refers to the current instance of the object.
        self.orders = orders #.copy()     # Stores a copy of the input DataFrame to avoid changing the original 
        ##self.monthly_revenue = None     #Initializes a placeholder attribute. We’ll later store computed monthly revenue here.
        
    def get_monthly_revenue(self):
        self.orders['order_date'] = pd.to_datetime(self.orders['order_date'])       #order date to date time
        self.orders['month'] = self.orders['order_date'].dt.month         #Extract the month number and create a new 'month' column
        self.orders = self.orders.sort_values(by='order_date')            #Sort the orders by 'order_date' to ensure correct chronological order
            
        self.monthly_revenue = (
            self.orders.groupby('month')                       #group by month
            .agg(monthly_revenue=('order_amount', 'sum'))      #sum of amount to get current reveue
            .reset_index()
        )
        self.monthly_revenue.rename(columns={'order_date': 'month'}, inplace = True)
            
        return self.monthly_revenue
        
        
        
    def calculate_revenue_growth(self):            
        if self.monthly_revenue is None:  #If the revenue hasn’t been calculated yet, it runs the get_monthly_revenue() method.
            self.get_monthly_revenue()
            
        self.monthly_revenue['previous_month_revenue'] = self.monthly_revenue['monthly_revenue'].shift(1).fillna(0) #lag monhtly reveue to get previous month revenue
        self.monthly_revenue['revenue_growth_percent'] = ((self.monthly_revenue['monthly_revenue'] -  self.monthly_revenue['previous_month_revenue'])/
        self.monthly_revenue['previous_month_revenue']).round(2) *100
            
        return self.monthly_revenue #_revenue
        


In [4]:
orders = pd.DataFrame({'order_date':['2023-01-05','2023-01-15','2023-02-20','2023-03-10','2023-03-22'],
                        'order_amount':[100, 150, 200, 250, 300]
                      })

analyzer = RevenueAnalyzer(orders)
monthly = analyzer.get_monthly_revenue()
growth = analyzer.calculate_revenue_growth()
print(growth)

   month  monthly_revenue  previous_month_revenue  revenue_growth_percent
0      1              250                     0.0                     inf
1      2              200                   250.0                   -20.0
2      3              550                   200.0                   175.0
